<a href="https://colab.research.google.com/github/beltrewilton/cx_analyzer/blob/main/Topic_Modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Grupo 4  - Proyecto `luzia` 
## Descubrir Tópicos basado en la técnica Latent Dirichlet Allocation [paper](https://www.jmlr.org/papers/volume3/blei03a/blei03a.pdf)
### Está aplicado utilizando la base de imdb previamente limpia para ir directo al punto.

In [8]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [9]:
from google.colab import files

uploaded = files.upload()

Saving banca_respuestas.csv to banca_respuestas (1).csv


In [10]:
from google.colab import files

uploaded = files.upload()

Saving stopwords_spanish.txt to stopwords_spanish.txt


In [11]:
def get_stopwords_list(stop_file_path):
    """load stop words """
    
    with open(stop_file_path, 'r', encoding="utf-8") as f:
        stopwords = f.readlines()
        stop_set = set(m.strip() for m in stopwords)
        return list(frozenset(stop_set))

In [13]:
stop_spanish = get_stopwords_list("stopwords_spanish.txt")
stop_spanish

stop_spanish.extend(['si','ahi','ahí', 'ah'])

In [20]:
## read the data, separated ith ";"

data = pd.read_csv("banca_respuestas.csv", sep = ";",
                   encoding_errors="ignore", encoding = 'utf_8')

data_tr = data.copy()

In [ ]:
data_tr = data_tr.dropna()
data_tr['razones'] = data_tr['razones'].str.lower()
data_tr['razones'] = data_tr['razones'].replace(',',"")
data_tr['razones'] = data_tr['razones'].replace('.',"")
data_tr['razones'] = data_tr['razones'].replace(';',"")
data_tr

In [23]:
count = CountVectorizer(stop_words= stop_spanish, max_df=0.1, max_features=5000)

In [ ]:
X = count.fit_transform(data_tr['razones'].values)

Más información de la implementacion Dirichlet en :

https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html

In [ ]:
# Esta tarea NLP para descubrir los tópicos consume tiempo, paciencia y café!
n_comp = np.where(len(data. index) < 500, 8,
                  np.where(len(data. index) < 1000, 10,
                           np.where(len(data. index) < 2000, 12,
                                    np.where(len(data. index) < 3000, 15, 20))))
lda = LatentDirichletAllocation(n_components=n_comp, random_state=123, learning_method='batch')
X_topics = lda.fit_transform(X)

A continuación se hace selección de los top 5 tópicos por 10 componentes encontrados.

In [ ]:
n_top_words = 10
feature_names = count.get_feature_names_out()
# for topic_idx, topic in enumerate(lda.components_, 1):
#     print(f'Topic {topic_idx}')
#     print(' '.join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))

# **First Stop!!!** Customer must review the topics and assign a name for the identified patterns!

Que sigue?
Mi recomendación es que miremos una implementación aún más robusta de Dirichlet sobre Deep Learning desarrollada por la empresa Uber llamada Pyro, más información:

*   https://medium.com/@anmolmittal_28682/evaluate-probabilistic-topic-models-pyro-latent-dirichlet-allocation-b84b73d110a7
*   https://pyro.ai/examples/intro_long.html




In [ ]:
topics = pd.DataFrame(lda.transform(X))
topics_encode = pd.DataFrame(np.where(topics >= 0.3, 1, 0))
other = pd.DataFrame(np.where(topics_encode.sum(axis = 1) == 0, 1, 0)) 
other.columns = ["other"]

In [ ]:
names = []
for topic_idx, topic in enumerate(lda.components_, 1):
    names.append(f'Topic {topic_idx}')

topics_encode.columns = names

In [ ]:
full_data = data.join(topics_encode) 
full_data = full_data.join(other)

full_data